In [ ]:
%load_ext autoreload
%autoreload 2
import sys

! python3 -m pip install  "sep_plot @ git+https://github.com/SEP-software/sep-plot.git@6331a1e36d8e3cdb4cfbc3539f31bdad1eb465a7" 


In [ ]:
!python -m pip install "sep_plot @ git+http://zapad.stanford.edu/bob/pySepPlot.git@2bffacb9fb36963339a0834c2b04a0aedff91db4"

In [ ]:
import copy
import numpy as np
from sep_python import FloatVector,get_sep_vector
class boxFilter(FloatVector):
    """Class for defining a filter on a box"""
    def __init__(self,sh,mask,zero_lag,vals=None,space_only=False):
        """
        Create a filter that is a n-d cube with some of the fixed (probably zeroed
        
        sh   - N-d arry of shape
        zero_lag - Location of the zero lag in the filter
        
        mask - Mask for filter coefs n-d array (should be 0 or 1
        vals - Value for filter coeficients
        
        space_only - Whether or not actually have storage
        """
        if tuple(sh)!= mask.shape:
            raise Exception(f"Mask {mask.shape} and filter {sh} shape must be the same")
        
        if vals is not None:
            if vals.shape != sh:
                raise Exception("vals not the same shape")
        sh=np.asarray(sh)[::-1]
        hyper=Hypercube.set_with_ns(sh)
        super().__init__(hyper,vals=vals)    
        self.mask=copy.deepcopy(mask)   
        self.zeroL=np.asarray(zero_lag)
        
    @classmethod
    def PEF(self,sh,one_loc):
        if len(sh) != len(one_loc):
            raise Exception("One location is not the same size as box filter shape")
        b=1
        one_1d=0
        for filt_s, one_s in zip(reversed(sh),reversed(one_loc)):
            if one_s < 0 or one_s >= filt_s:
                raise Exception("Illegal locaiton for one in the filter")
            one_1d+=one_s*b
            b*=filt_s
        
        msk=np.ones(sh,dtype=np.float32)
        m=np.ravel(msk)
        vals=np.zeros(sh,dtype=np.float32)
        v=np.ravel(vals)
        v[:one_1d]=0
        m[:one_1d+1]=0
        v[one_1d]=1
        return boxFilter(sh,msk,one_loc,vals=vals)

                     
    def clone(self):
        """Return a clone of the helix filter"""
        x=boxFilter(self.get_nd_array().shape,self.mask,self.zeroL,vals=self.get_nd_array())
        return x
    
    def cloneSpace(self):
        x=boxFilter(self.get_nd_array().shape,self.mask,self.zeroL,space_only=True)
        return x


In [ ]:
import numba
from generic_solver._pyOperator import Operator
class convOpAdjData(Operator):
    def __init__(self,model,data,filt):
        if not model.checkSame(data) and not isinstance(model,FloatVector):
            raise Exception("Expecting model and data to be the same shape and float vectors")
        self.filt=filt.clone()
        
        if len(filt.get_nd_array().shape) >len(model.get_nd_array().shape):
            raise Exception("Filter must be same number of dimensions or smaller than model/data")
        super().__init__(model,data)
        
        
    
    def forward(self,add,model,data):
        self.checkDomainRange(model,data)
        if not add:
            data.zero()
        m=model.get_nd_array()
        d=data.get_nd_array()
        f=self.filt.get_nd_array()
        if len(m.shape)==2:
            if len(f.shape)==2:
                forward_2_2_m(m,f,d,self.filt.zeroL)
            else:
                forward_2_1_m(m,f,d,self.filt.zeroL)
        else:
            forward_1_1_m(m,f,d,self.filt.zeroL)
        
        
    
    def adjoint(self,add,model,data):
        self.checkDomainRange(model,data)
        if not add:
            model.zero()
        m=model.get_nd_array()
        d=data.get_nd_array() 
        f=self.filt.get_nd_array()
        
        if len(m.shape)==2:
            if len(f.shape)==2:
                adjoint_m_2_2(m,f,d,self.filt.zeroL)
            else:
                adjoint_m_2_1(m,f,d,self.filt.zeroL)
        else:
            adjoint_m_1_1(m,f,d,self.filt.zeroL)
        
class convOpAdjFilter(Operator):
    def __init__(self,filt,data,model):
        if not model.checkSame(data) and not isinstance(model,FloatVector):
            raise Exception("Expecting model and data to be the same shape and float vectors")
        self.model=copy.deepcopy(model)
        
        if len(filt.get_nd_array().shape) >len(model.get_nd_array().shape):
            raise Exception("Filter must be same number of dimensions or smaller than model/data")
        super().__init__(filt,data)
        
        
    
    def forward(self,add,filt,data):

        self.checkDomainRange(filt,data)
        if not add:
            data.zero()
        m=self.model.get_nd_array()
        d=data.get_nd_array()
        ftemp=filt.clone()
        f=ftemp.get_nd_array()
        f[:]=f[:]*filt.mask[:]

        if len(m.shape)==2:
            if len(f.shape)==2: 
               forward_2_2(m,f,d,filt.zeroL)
            else:
                forward_2_1(m,f,d,filt.zeroL)
        else:
            forward_1_1(m,f,d,filt.zeroL)
        
        
    def adjoint(self,add,filt,data):
        self.checkDomainRange(filt,data)
        if not add:
            filt.zero()
        m=self.model.get_nd_array()
        d=data.get_nd_array() 
        ftemp=filt.clone()
        f=ftemp.get_nd_array()
        ftemp.zero()
        if len(m.shape)==2:
            if len(f.shape)==2:
                adjoint_f_2_2(m,f,d,filt.zeroL)
            else:
                adjoint_f_2_1(m,f,d,filt.zeroL)
        else:
            adjoint_f_1_1(m,f,d,filt.zeroL)
        f[:]=f[:]*filt.mask[:]
        filt.scale_add(ftemp)
@numba.njit()
def forward_2_2_m(m ,f,d,zero): 
    for i2 in range(m.shape[0]):
        if i2 < zero[0] or i2 >m.shape[0]-zero[0]:
            d[i2,:]+=m[i2,:]
        else:
            for i1 in range(m.shape[1]):
                if i1<zero[1] or i1 > m.shape[1]-zero[1]:
                    d[i2,i1]+=m[i2,i1]
                else:
                    for if2 in range(0,f.shape[0]):
                        for if1 in range(0,f.shape[1]):
                            d[i2,i1]+=f[if2,if1]*m[i2-if2+zero[0] ,i1-if1+zero[1]]

    
@numba.njit()
def forward_2_1_m(m,f,d,zero):               
    for i2 in range(m.shape[0]):
        for i1 in range(m.shape[1]):
            if i1 < zero[0] or i1 >m.shape[0]-zero[0]:
                d[i2,i1]+=m[i2,i1]
            else:
                for if1 in range(0,f.shape[1]):
                    d[i2,i1]+=f[if1]*m[i2 ,i1-if1+zero[0]]
                                 
@numba.njit()
def forward_1_1_m(m,f,d,zero):
    for i1 in range(m.shape[0]):
        if i1 < zero[0] or i1 >m.shape[0]-zero[0]:
            d[i1]+=m[i1]
        else:
            for if1 in range(0,f.shape[1]):
                d[i1]+=f[if1]*m[i1-if1+zero[0]]
                
@numba.njit()
def forward_2_2(m,f,d,zero): 
    for i2 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
        for i1 in range(f.shape[1]-zero[1]-1,m.shape[1]-zero[1]):
            for if2 in range(0,f.shape[0]):
                for if1 in range(0,f.shape[1]):
                    d[i2,i1]+=f[if2,if1]*m[i2-if2+zero[0] ,i1-if1+zero[1]]

    
@numba.njit()
def forward_2_1(m,f,d,zero):               
    for i2 in range(m.shape):
        for i1 in range(f.shape[0]-zero[0]-1,m.shape[1]-zero[0]):
                for if1 in range(0,f.shape[1]):
                    d[i2,i1]+=f[if1]*m[i2 ,i1-if1+zero[0]]
                                 
@numba.njit()
def forward_1_1(m,f,d,zero):             
    for i1 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
            for if1 in range(0,f.shape[1]):
                d[i1]+=f[if1]*m[i1-if1+zero[0]]

@numba.njit()
def adjoint_m_2_2(m,f,d,zero):          
    for i2 in range(m.shape[0]):
        if i2 < zero[0] or i2 >m.shape[0]-zero[0]:
            m[i2,:]+=d[i2,:]
        else:
            for i1 in range(m.shape[1]):
                if i1<zero[1] or i1 > m.shape[1]-zero[1]:
                    m[i2,i1]+=d[i2,i1]
                else:
                    for if2 in range(0,f.shape[0]):
                        for if1 in range(0,f.shape[1]):
                            m[i2-if2+zero[0] ,i1-if1+zero[1]]+=d[i2,i1]*f[if2,if1]

@numba.njit()
def adjoint_m_2_1(m,f,d,zero):  
    for i2 in range(m.shape[0]):
        for i1 in range(m.shape[1]):
            if i1 < zero[0] or i1 >m.shape[0]-zero[0]:
                m[i2,i1]+=d[i2,i1]
            else:
                for if1 in range(0,f.shape[0]):
                    m[i2,i1-if1+zero[1]]+=d[i2,i1]*f[if1]
  
@numba.njit()
def adjoint_m_1_1(m,f,d,zero):
    for i1 in range(m.shape[0]):
        if i1 < zero[0] or i1 >m.shape[0]-zero[0]:
            m[i1]+=d[i1]
        else:
            for if1 in range(0,f.shape[0]):
                m[i1-if1+zero[1]]+=d[i1]*f[if1]
                
@numba.njit()
def adjoint_f_2_2(m,f,d,zero):    

    for i2 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
        for i1 in range(f.shape[1]-zero[1]-1,m.shape[1]-zero[1]):
            for if2 in range(0,f.shape[0]):
                for if1 in range(0,f.shape[1]):
                    f[if2,if1]+=m[i2-if2+zero[0] ,i1-if1+zero[1]]*d[i2,i1]
@numba.njit()
def adjoint_f_2_1(m,f,d,zero):  
    for i2 in range(m.shape[0]):
        for i1 in range(f.shape[0]-zero[0]-1,m.shape[1]-zero[0]):
                for if1 in range(0,f.shape[0]):
                    f[if1]+=m[i2,i1-if1+zero[1]]*d[i2,i1]
  
@numba.njit()
def adjoint_f_1_1(m,f,d,zero):
    for i1 in range(f.shape[0]-zero[0]-1,m.shape[0]-zero[0]):
            for if1 in range(0,f.shape[0]):              
                f[if1]+=m[i1-if1+zero[1]]*d[i1]

In [ ]:
class weight_op(Operator):
    def __init__(self,model,data,wt):
        w=np.copy(wt)
        self._wt=w.ravel()
        super().__init__(model,data)
    
    def forward(self,add,model,data):
        self.checkDomainRange(model,data)
        if not add:
            data.zero()
        d2=data.get_nd_array()
        d=d2.ravel()
        m2=model.get_nd_array()
        m=m2.ravel()
        d[:]=d[:]+m[:]*self._wt[:]
    def adjoint(self,add,model,data):
        self.checkDomainRange(model,data)
        if not add:
            model.zero()
        d2=data.get_nd_array()
        d=d2.ravel()
        m2=model.get_nd_array()
        m=m2.ravel()
        m[:]=m[:]+d[:]*self._wt[:]      


In [ ]:
from generic_solver import ProblemL2Linear,ProblemL2LinearReg
from generic_solver import  LCGsolver, BasicStopper
from generic_solver._pyOperator import ChainOperator
from sep_plot import Grey



@numba.njit()
def find_zeros_2d(sh,zero,m,wt):
    for i2 in range(sh[0]-zero[0]-1,m.shape[0]-zero[0]):
        for i1 in range(sh[1]-zero[1]-1,m.shape[1]-zero[1]):
            haveZero=False
            for if2 in range(0,sh[0]):
                for if1 in range(0,sh[1]):
                    if m[i2-if2+zero[0] ,i1-if1+zero[1]]==0.:
                        haveZero=False
            wt[i2,i1]=1



            
def findPef(model,sh,zero):
    filt=boxFilter.PEF(sh,zero)
    data=model.clone()
    wtVec=data.clone()
    wtVec.zero()
    find_zeros_2d(sh,zero,model.get_nd_array(),wtVec.get_nd_array())
    wt=weight_op(model,model,wtVec.get_nd_array())
    op=convOpAdjFilter(filt,data,data)
    data.scale(-1.)
    data.get_nd_array()[:,:]=np.multiply(data.get_nd_array(),wtVec.get_nd_array())
    wt_op=ChainOperator(op,wt)
    prob=ProblemL2Linear(filt,data,wt_op)
    stop=BasicStopper(niter=1000)
    solve=LCGsolver(stop)
    solve.run(prob,verbose=True)
    return prob.model

In [ ]:
!wget https://raw.githubusercontent.com/rgclapp007/gp211-class-notebooks/main/data/seabin.HH

In [ ]:
import numpy as np
from sep_python import Hypercube, default_io
vec=default_io.vector_from_storage("seabin.HH")
import holoviews as hv
hv.extension('bokeh','matplotlib')
Grey(vec)

In [ ]:
sh=(5,9)
zero=(0,3)
pef=findPef(vec,sh,zero)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
known=vec.clone()
@numba.njit()
def known_data(vec,known):
    for i2 in range(vec.shape[0]):
        for i1 in range(vec.shape[1]):
            if vec[i2,i1]==0:
                known[i2,i1]=0
            else:
                known[i2,i1]=1
known_data(vec.get_nd_array(),known.get_nd_array())

Grey(known)

In [ ]:


j_op=weight_op(vec,vec,known.get_nd_array())
problemStop=BasicStopper(niter=500)
data=vec.clone()
d=data.get_nd_array()

model=data.clone()
reg=convOpAdjData(model,data,pef)
eps=1000

problemconv=ProblemL2LinearReg(model,data,j_op,eps,reg_op=reg)
solve_base=LCGsolver(problemStop)
solve_base.run(problemconv)

Grey(problemconv.model,pclip=99)+Grey(data,pclip=100)

In [ ]:
x=vec.clone()
reg_1.forward(False,vec,x)
Grey(x)

In [ ]:
from sep_plot import Graph
Graph(vec.get_nd_array()[0,:])

In [ ]:
class randomRes(ProblemL2LinearReg):
    """Linear inverse problem regularized of the form 1/2*|Lm-d|_2 + epsilon^2/2*|Am-m_prior|_2"""

    def __init__(
        self,
        model,
        data,
        op,
        epsilon,
        randLevel=.6,
        grad_mask=None,
        reg_op=None,
        prior_model=None,
        prec=None,
        minBound=None,
        maxBound=None,
        boundProj=None,
        
    ):
        super().__init__( model,data,op,epsilon,grad_mask,reg_op,prior_model,prec,minBound,maxBound,boundProj)
        self.rnd=self.res.vecs[1].clone()
        x=self.rnd.get_nd_array().ravel()
        x[:]=np.random.normal(0,randLevel,size=(self.rnd.get_hyper().get_n123()))
    def resf(self, model):
        """Method to return residual vector r = [r_d; r_m]: r_d = Lm - d; r_m = epsilon * (Am - m_prior)"""
        if model.norm() != 0.0:
            self.op.forward(False, model, self.res)
        else:
            self.res.zero()
        # Computing r_d = Lm - d
        self.res.vecs[0].scaleAdd(self.data, 1.0, -1.0)
        self.res.vecs[1].scaleAdd(self.rnd,1,-1.)
        # Computing r_m = Am - m_prior
        if self.prior_model is not None:
            self.res.vecs[1].scaleAdd(self.prior_model, 1.0, -1.0)
        # Scaling by epsilon epsilon*r_m
        self.res.vecs[1].scale(self.epsilon)
        return self.res

In [ ]:
problemStop=BasicStopper(niter=400)
eps=1000

model.zero()
model=data.clone()
reg=convOpAdjData(model,data,pef)
problemconv2=randomRes(model,data,j_op,eps,randLevel=.07,reg_op=reg)
solve_base=LCGsolver(problemStop)
print(problemconv2.res.vecs[1].get_nd_array()[0,:10])
solve_base.run(problemconv2)

Grey(problemconv2.model,pclip=99)+Grey(data,pclip=100)+Grey(problemconv2.res.vecs[1])

In [ ]:
vecs=[]
for i in range(9):
    reg=convOpAdjData(model,data,pef)
    problemconv2=randomRes(model,data,j_op,eps,randLevel=.07,reg_op=reg)
    solve_base=LCGsolver(problemStop)
    solve_base.run(problemconv2)
    vecs.append(problemconv2.model.clone())


In [ ]:
def plot_result_reg(item):
    return Grey(vecs[item])
# When run live, this cell's output should match the behavior of the GIF below
dmap = hv.DynamicMap(plot_result_reg, kdims=['iter'])
dmap.redim.range(iter=(0,len(vecs)))